In [20]:
import pandas as pd

### Ordenar las subseries de forma descendente

La finalidad es ordenar de forma descendente las subseries 1..n que hay en la serie.

Output solicitado:

[6,5,4,3,2,1,4,3,2,1,5,4,3,2,1,2,1,9,8,7,6,5,4,3,2,1]

In [21]:
serie = pd.Series([1,2,3,4,5,6,1,2,3,4,1,2,3,4,5,1,2,1,2,3,4,5,6,7,8,9])
serie

0     1
1     2
2     3
3     4
4     5
5     6
6     1
7     2
8     3
9     4
10    1
11    2
12    3
13    4
14    5
15    1
16    2
17    1
18    2
19    3
20    4
21    5
22    6
23    7
24    8
25    9
dtype: int64

## Solución

In [22]:
filtro = [serie > serie.shift()]
pd.DataFrame((serie.groupby(filtro).cumcount())[serie.groupby(filtro).groups[False]]
             ,index=range(0,len(serie))).fillna(method='ffill').groupby([0]).cumcount(ascending=False).add(1)

0     6
1     5
2     4
3     3
4     2
5     1
6     4
7     3
8     2
9     1
10    5
11    4
12    3
13    2
14    1
15    2
16    1
17    9
18    8
19    7
20    6
21    5
22    4
23    3
24    2
25    1
dtype: int64

### Solucion por pasos

Primero creao un filtro que generar 2 grupos, TRUE y FALSE para los que cumplan la condicion del filtro.

In [23]:
serie.groupby([serie > serie.shift()]).groups

{False: Int64Index([0, 6, 10, 15, 17], dtype='int64'),
 True: Int64Index([1, 2, 3, 4, 5, 7, 8, 9, 11, 12, 13, 14, 16, 18, 19, 20, 21, 22, 23,
             24, 25],
            dtype='int64')}

Con cumcount() enumera ambos grupos (True y False), por lo que tengo el grupo 'False' que marca el inicio de cada subgrupo una enumeracion enumerado.

In [26]:
si = serie.groupby([serie > serie.shift()]).groups[False]
s = serie.groupby([serie > serie.shift()]).cumcount()

s[si]

0     0
6     1
10    2
15    3
17    4
dtype: int64

Ahora extiendo la serie en una DataFrame con indices consecutivos y relleno los valores con el numero del grupo.

In [27]:
df = pd.DataFrame(s[si],index=range(0,len(serie)))
df

,0
0,0.0
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,1.0
7,NaN
8,NaN
9,NaN


In [28]:
df = df.fillna(method='ffill')
df

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,1.0
7,1.0
8,1.0
9,1.0


Si volvermos a agrupar por los valores, ya tenemos las subseries, pero con el mismo valor para cada serie.

In [29]:
df.groupby([0]).groups

{0.0: Int64Index([0, 1, 2, 3, 4, 5], dtype='int64'),
 1.0: Int64Index([6, 7, 8, 9], dtype='int64'),
 2.0: Int64Index([10, 11, 12, 13, 14], dtype='int64'),
 3.0: Int64Index([15, 16], dtype='int64'),
 4.0: Int64Index([17, 18, 19, 20, 21, 22, 23, 24, 25], dtype='int64')}

Hacemos el cumcount de nuevo en descendente

In [30]:
df.groupby([0]).cumcount(ascending=False)

0     5
1     4
2     3
3     2
4     1
5     0
6     3
7     2
8     1
9     0
10    4
11    3
12    2
13    1
14    0
15    1
16    0
17    8
18    7
19    6
20    5
21    4
22    3
23    2
24    1
25    0
dtype: int64

Y agregamos un 1 para cuadrar los valores

In [31]:
df.groupby([0]).cumcount(ascending=False).add(1)

0     6
1     5
2     4
3     3
4     2
5     1
6     4
7     3
8     2
9     1
10    5
11    4
12    3
13    2
14    1
15    2
16    1
17    9
18    8
19    7
20    6
21    5
22    4
23    3
24    2
25    1
dtype: int64